In [1]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

example_sce = anndata.read_h5ad(save_path)
example_sce

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

In [2]:
from scdesigner.experimental.estimators import negative_binomial_regression
from scdesigner.experimental.transform import nullify, amplify, data_frame_mask

params = negative_binomial_regression(example_sce, "~ bs(pseudotime, degree=6)", epochs=100)
outcomes = example_sce.var_names[:4]
mask = data_frame_mask(params["coefficient"], ["pseudotime"], outcomes)
null_params = nullify(params, "coefficient", mask)
null_params["coefficient"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,2.197865,2.268209,1.318629,1.286049,3.840294,0.448331,-0.349397,2.105087,1.068234,1.211985,...,0.552880,0.290994,1.440957,-0.160437,1.048037,-1.233168,-0.637515,1.724128,-2.435935,-0.178139
"bs(pseudotime, degree=6)[1]",0.000000,0.000000,0.000000,0.000000,3.003995,-15.069673,-1.397554,-10.418838,-11.106424,-13.283100,...,-1.553560,-3.440922,-1.226781,1.312675,0.608415,3.222896,3.997243,-0.758272,-5.780035,-3.938097
"bs(pseudotime, degree=6)[2]",0.000000,0.000000,0.000000,0.000000,-3.633759,4.878798,6.335586,1.287712,10.443920,-2.103594,...,3.010540,2.766718,3.452707,4.106813,0.911357,-0.469060,3.836551,1.942890,-2.752351,3.260376
"bs(pseudotime, degree=6)[3]",0.000000,0.000000,0.000000,0.000000,-14.695806,5.241379,5.936082,-6.154892,-7.455820,-6.045755,...,1.158814,2.801613,0.697978,1.391438,-4.358403,4.425304,0.441013,-1.976071,-3.182852,4.897079
"bs(pseudotime, degree=6)[4]",0.000000,0.000000,0.000000,0.000000,-1.076773,2.775726,2.266021,-8.846365,-4.309461,-8.769652,...,-0.812769,-1.884889,-4.701144,-2.436549,-1.081311,3.203225,-0.545738,-5.097927,-4.957916,-1.141697
"bs(pseudotime, degree=6)[5]",0.000000,0.000000,0.000000,0.000000,-5.658473,1.579606,-1.084344,1.496725,3.157530,2.442446,...,1.184013,2.084829,0.848783,0.073521,0.972107,0.687638,2.092434,-1.340444,-3.438744,-1.626804
"bs(pseudotime, degree=6)[6]",0.000000,0.000000,0.000000,0.000000,-4.979625,2.345560,1.942488,3.060781,3.689172,3.768801,...,1.268168,-0.480219,-1.241686,-0.989558,1.673595,1.211747,0.363372,-2.069894,6.488352,-0.483170


In [3]:
from scdesigner.experimental.samplers import negative_binomial_regression_sample

samples = negative_binomial_regression_sample(params, example_sce.obs, "~ bs(pseudotime, degree=6)")
samples.X[:10, :10]

array([[ 65,   5,  21,  12,   0,  21,   7,   0,   1,   0],
       [  0,   0,   3,   0,   0,   8,  25,   0,   0,   0],
       [ 37,   6,  48,  56,   0,  16,  10,   1,   3,   0],
       [ 18,  36,  27,  51,   0,   6,   0,  11,  30,  14],
       [  1,   0,   2,   0,   0,   0,  21,   0,   0,   0],
       [  2,   0,   0,   0,   8,   0,   9,   0,   2,   0],
       [  7,   3,   1,   0,  84,   0,   0,   2,   0,   0],
       [ 78,   2,  35,  11,   0,  23,   6,   1,   0,   0],
       [308, 116,  53,  27,   1,   2,   0,   2,   2,   1],
       [ 10,   0,  53,  24,   0,  15,   4,   1,   0,   0]])

In [4]:
from scdesigner.experimental.diagnose import compare_means, compare_standard_deviation, compare_umap
import numpy as np

log_relative = lambda x: np.log1p(x / x.sum(axis=1, keepdims=True))
compare_means(example_sce, samples, log_relative)


alt.LayerChart(...)

In [5]:
compare_standard_deviation(example_sce, samples, log_relative)

alt.LayerChart(...)

In [6]:
compare_umap(example_sce, samples)

alt.FacetChart(...)

In [7]:
from scdesigner.experimental.estimators import negative_binomial_copula

params = negative_binomial_copula(example_sce, "~ bs(pseudotime, degree=2)", epochs=10)
mask = data_frame_mask(params["covariance"], outcomes)
null_params = nullify(params, "covariance", mask)
null_params["covariance"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Pyy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Iapp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chgb,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Rbp4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Spp1,0.390316,0.805238,-0.277111,0.299423,2.398408,-0.224031,-0.901790,0.563527,0.416978,0.324764,...,-0.086490,-0.082385,-0.006725,-0.203580,0.395808,-0.024248,-0.108033,0.026309,-0.108967,0.004930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ffar2,0.006158,0.028478,0.231020,-0.003963,-0.024248,0.132013,0.000332,0.028941,0.025344,0.045021,...,0.025279,0.090399,-0.043960,0.028697,0.004414,0.837155,-0.007297,-0.019008,-0.018677,0.057463
Hes6,0.321812,0.647688,-0.102723,0.172212,-0.108033,-0.077800,-0.043403,0.383621,0.379064,0.357346,...,0.154016,-0.019688,0.418672,0.429478,0.174300,-0.007297,1.163314,0.355314,-0.024987,0.082916
Serpinh1,0.453570,0.651136,-0.050421,0.286822,0.026309,0.014142,-0.161568,0.388464,0.374632,0.379272,...,0.082182,-0.007747,0.291038,0.370178,0.148916,-0.019008,0.355314,1.062771,-0.028310,0.090869
Npy,-0.417389,-0.248446,-0.143492,-0.299228,-0.108967,-0.099273,0.223233,0.072348,-0.022724,0.044870,...,0.087685,-0.017850,0.014516,-0.052802,0.055578,-0.018677,-0.024987,-0.028310,0.432653,-0.024307


In [8]:
mask = data_frame_mask(params["coefficient"], ["pseudotime"], outcomes)
null_params = amplify(params, "coefficient", mask, factor=2)
null_params["coefficient"] / params["coefficient"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"bs(pseudotime, degree=2)[1]",2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"bs(pseudotime, degree=2)[2]",2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [9]:
from scdesigner.experimental.samplers import negative_binomial_copula_sample

negative_binomial_copula_sample(params, example_sce.obs, "~ bs(pseudotime, degree=2)")

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'